In [1]:
import morePVs_output as opm
import morePVs as mpv
import os
import en_utilities as um
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D



Plotting SCM and SSM / vs kWp and kWh storage
----------------------------------------------------------------
Site J


study_`combined_siteJ_bat5`
Single set of parametetrs:

 


In [2]:
# -------------------------------
# combine  input and result files
# -------------------------------
# Change to bat2
project='EN1a_pv_bat2'
study_name='siteJ_bat2_2'
base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\studies'
op = opm.Output(base_path = base_path,
                          project = project,
                          study_name = study_name)
self = op
df = self.data.copy()
df_in = self.study_parameters.copy()
df = df.merge(df_in,left_index = True, right_index=True)
for c in df.columns:
    if '_x' in c:
        nc = c[:-2]
        df.rename(columns={c:nc}, inplace=True)
    if '_y' in c:
        df= df.drop([c],axis=1)
df.head()
arrangement = 'cp_only'
fontsize=12


In [3]:
# Directories for plots
# ---------------------
plot_bau = os.path.join(op.plot_path,'ssc_ssm')
if not os.path.exists(plot_bau):
    os.makedirs(plot_bau)
plot_en = os.path.join(op.plot_path,'ssc_ssm')
if not os.path.exists(plot_en):
    os.makedirs(plot_en)

In [4]:
# ---------------
# Organise labels
# ---------------
df.loc[:,'site'] = df.loc[:,'load_folder'].apply(lambda x : x[-1])
df.loc[:, 'labels'] = df.loc[:,'site']
floors = {'A':12,'E':7, 'D':9,'B':8,'H':3,'I':4,'G':4,'C':4,'J':4,'F':5}
numhouses={'A':208,'E':161, 'D':138,'B':104,'H':52,'I':48,'G':44,'C':34,'J':26,'F':20}
labels ={}
for s, f in floors.items():
    u = numhouses[s]
    labels[s] = s +'('+str(u)+'/'+str(f)+')'
# print(labels)
df.loc[:, 'label'] = df.loc[:, 'site'].apply(lambda x: labels[x])

# ----------------
# Create Site tags
# ----------------
site_tag ={'A': 'a208_f12_cp34',
 'B': 'a104_f8_cp57',
 'C': 'a34_f4_cp34',
 'D': 'a138_f9_cp45',
 'E': 'a161_f7_cp38',
 'F': 'a20_f5_cp37',
 'G': 'a44_f4_cp17',
 'H': 'a52_f3_cp27',
 'I': 'a48_f4_cp09',
 'J': 'a26_f4_cp44'}

# ----------------
# Set up constants
# ----------------
# NB Some of these are now superfluous
sites=['A','E','D','B','H','I','G','C','J','F']
# Plotting set-up parameters
cmap = mpl.cm.seismic
alpha=1

markers =['x','o','D','*','^','H','s','v','>','<']
site_markers = dict(zip(sites,markers))

colours = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
site_colours = dict(zip(sites,colours))

a_terms =df.a_term.tolist()
term_markers = dict(zip(a_terms,markers))
# dashList = [(5,2),(2,5),(4,10),(3,3,2,2),(5,2,20,2)] 
# term_dashes = dict(zip(a_terms,dashList))

# get pv sizes for maximum pv systems
# -----------------------------------
maximums={}
pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
pv_ref=pd.read_csv(pv_ref_file)
pv_ref = pv_ref.set_index('pv_cap_id')
for i in pv_ref.index:
    if 'max' in i and 'site' in i:
        site = um.find_between(i,'_','_')
        maximums[site] = pv_ref.loc[i,'kW']
# ----------------------------------
# get lists of parameters to iterate
# ----------------------------------
en_capexes= [c for c in ['capex_low','capex_med','capex_high']if c in df.en_capex_id.tolist()]
# parents = [p for p in ['EA310_12c_FIT8', 'EA310_12c_FIT12', 'EA310_12c',\
#            'EA310_10c_FIT8', 'EA310_10c_FIT12', 'EA310_10c',\
#          'EA310_8c_FIT8', 'EA310_8c_FIT12','EA310_8c'] if p in df.parent.tolist()]
parents =df.parent.drop_duplicates().tolist()

In [5]:

# get kWp / unit , total $ / unit  and % bau for each scenario
# ------------------------------------------------------------
df['pv_filename'].fillna('zero',inplace=True)

for s in df.index:
    if df['cp_battery_id'].fillna('no_battery').loc[s] == 'no_battery':
        df.loc[s,'cp_battery_kWh']=0.0
    else:
        df.loc[s,'cp_battery_kWh'] = np.float(df.loc[s,'cp_battery_id'].rpartition('_')[-1])


for s in df.index:

        site = df.loc[s,'load_folder'][-1]
        df.loc[s,'site']=site
        if df.loc[s,'pv_filename'] == 'zero':
            df.loc[s,'kWp_unit'] =0
        elif 'max' in df.loc[s,'pv_filename']:
            df.loc[s,'kWp_unit']= maximums[site]/df.loc[s,'number_of_households']
        else:
            df.loc[s,'kWp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10
        df.loc[s,'$_unit'] = df.loc[s,'total$_building_costs_mean']/df.loc[s,'number_of_households']

        # Find delta_SC and delta_SS wrt scenario with same PV but zero storage
        base_scenario = df[(df['kWp_unit']==df.loc[s,'kWp_unit']) & \
                           (df['cp_battery_kWh'] == 0.0)].index[0]
        
        df.loc[s,'delta_sc'] = df.loc[s,'self-consumption_mean'] - df.loc[base_scenario,'self-consumption_mean']
        df.loc[s,'delta_ss'] = df.loc[s,'self-sufficiency_mean'] - df.loc[base_scenario,'self-sufficiency_mean']

#         df.loc[s,'improved_sc'] = df.loc[s,'delta_sc']/ df.loc[base_scenario,'self-consumption_mean'] * 100
#         df.loc[s,'improved_ss'] = df.loc[s,'delta_ss']/ df.loc[base_scenario,'self-sufficiency_mean'] * 100

        df.loc[s,'kWh_unit']= df.loc[s,'cp_battery_kWh']/df.loc[s,'number_of_households']



In [6]:
# ---------------------
# prep and plot - SSM - Self Sufficiency
# ---------------------
arrangement = 'cp_only'
strategy = df['cp_battery_strategy'].dropna().values[0]
                  
dfx=df.copy()

dfx.loc[:,'kWp_str'] = dfx.loc[:,'kWp_unit'].apply(lambda x : str(round(x,2)))
dfx.loc[:,'aterm_str'] = dfx.loc[:,'a_term'].apply(lambda x : str(round(x,0)))
dfx.loc[:,'kWh_str'] = dfx.loc[:,'kWh_unit'].apply(lambda x : str(round(x,2)))

# dfx['combined'] = dfx['site']+ '_' + dfx['arrangement'] + '_'+ dfx['kWp_str']+ '_'+(dfx['aterm_str']) + '_' + dfx.loc[:,'kWh_str']
#dfx = dfx.drop_duplicates('combined')
dfx['kWp_kWh'] =  dfx['kWp_str'] + '_'+ dfx['kWh_str']
dfx['new_label']= dfx['label']+ '_'+ dfx['kWp_str'] 

# ----------------------
# reindex and stack data
# ----------------------
dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
dfu = dfx['self-sufficiency_mean'].unstack()
cols =dfu.columns
dfu.columns=cols
dfv=dfu.transpose()
# NB - parent, capex combinations must have sufficient data
# if (dfv.shape[0] <= 1 or dfv.shape[1] <= 1):
#     continue

# ---------------------
# Plot SSM vs kW peak
# ---------------------
dfplot = dfv.copy()
cols = dfplot.columns
cols2 = [str(round(c,1)) + ' kWh / unit' for c in cols]
dfplot.columns = cols2
fig, ax = plt.subplots()
# Legend outside box and resize chart to fit
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
maxy=50
ax.set_ylim(0,55)
ax.set_ylabel('Self Sufficiency %')
ax.set_xlabel('PV kWp / unit')

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'ssm_kWp'  +'_'+ 'site_'  +site + '_'+ strategy +'.jpg'
plotFile = os.path.join(plot_bau,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()

# -----------------
# Plot SSM vs kWh 
# -----------------
dfplot = dfv.copy().T
cols = dfplot.columns
cols2 = [str(round(c,1)) + ' kWp / unit' for c in cols]
dfplot.columns = cols2
fig, ax = plt.subplots()
# Legend outside box and resize chart to fit
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
maxy=50
ax.set_ylim(0,55)
ax.set_ylabel('Self Sufficiency %')
ax.set_xlabel('Storage kWh / unit')

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'ssm_kWh'  +'_'+ 'site_' + site + '_'+ strategy +'.jpg'
plotFile = os.path.join(plot_bau,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()

In [7]:

# ------------------------------
# Plot 3D SSM vs kW peak and kWh 
# ------------------------------
dfplot = dfv.copy()
x = dfplot.columns
y = dfplot.index
X,Y = np.meshgrid(x,y)
Z = dfplot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z)

# Set Axes limits
# ---------------
ax.set_zlim(0,50)
maxz=50

#Set rotation angle 
#------------------
ax.view_init(azim=210)

# Label Axes
# ----------
ax.set_zlabel('Self-Sufficiency %', fontsize=fontsize)
ax.set_ylabel('PV kWp / unit',fontsize=fontsize)
ax.set_xlabel('Storage kWh / unit',fontsize=fontsize)

# Set Tick Lablels
# ----------------
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text2D(0,1.07,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text2D(0,1.0,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text2D(0,0.93,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'ssm_3D'  +'_'+ 'site_'  +site + '_'+ strategy + '_' + arrangement +'.jpg'
plotFile = os.path.join(plot_bau,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()

In [8]:
# ---------------------
# prep and plot SCM
# ---------------------
strategy = df['cp_battery_strategy'].dropna().values[0]

dfx=df.copy()
dfx.loc[:,'kWp_str'] = dfx.loc[:,'kWp_unit'].apply(lambda x : str(round(x,2)))
dfx.loc[:,'aterm_str'] = dfx.loc[:,'a_term'].apply(lambda x : str(round(x,0)))
dfx.loc[:,'kWh_str'] = dfx.loc[:,'kWh_unit'].apply(lambda x : str(round(x,2)))

#dfx['combined'] = dfx['site']+ '_' + dfx['arrangement'] + '_'+ dfx['kWp_str']+ '_'+(dfx['aterm_str']) + '_' + dfx.loc[:,'kWh_str']
#dfx = dfx.drop_duplicates('combined')
dfx['kWp_kWh'] =  dfx['kWp_str'] + '_'+ dfx['kWh_str']
dfx['new_label']= dfx['label']+ '_'+ dfx['kWp_str'] + '_'+ dfx['aterm_str']


# ----------------------
# reindex and stack data SCM
# ----------------------
dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
dfu = dfx['self-consumption_mean'].unstack()
cols =dfu.columns
dfu.columns=cols
dfv=dfu.transpose()


# --------------------
# Plot SCM vs kW peak
# --------------------
dfplot = dfv.copy()
cols = dfplot.columns
cols2 = [str(round(c,1)) + ' kWh / unit' for c in cols]
dfplot.columns = cols2
fig, ax = plt.subplots()
# Legend outside box and resize chart to fit
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
ax.set_ylim(25,105)
maxy=100
ax.set_ylabel('Self-Consumption %')
ax.set_xlabel('PV kWp / unit')

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'scm_kWp'  +'_'+ 'site_' + site + '_'+ strategy +'.jpg'
plotFile = os.path.join(plot_en,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()

    # ----------------
# Plot SCM vs kWh
# ----------------
dfplot = dfv.copy().T
cols = dfplot.columns
cols2 = [str(round(c,1)) + ' kWp / unit' for c in cols]
dfplot.columns = cols2
fig, ax = plt.subplots()
# Legend outside box and resize chart to fit
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
ax.set_ylim(25,105)
maxy=100
ax.set_ylabel('Self-Consumption %')
ax.set_xlabel('Storage kWh / unit')

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'scm_kWh'  +'_'+ 'site_' + site + '_'+ strategy +'.jpg'
plotFile = os.path.join(plot_en,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()


In [9]:
# ------------------------------
# Plot 3D SC vs kW peak and kWh 
# ------------------------------
dfplot = dfv.copy()
x = dfplot.columns
y = dfplot.index
X,Y = np.meshgrid(x,y)
Z = dfplot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z)

# Set Axes limits
# ---------------
ax.set_zlim(25,100)
maxz=100

#Set rotation angle 
#------------------
ax.view_init(azim=30)

# Label Axes
# ----------
ax.set_zlabel('Self-Consumption %', fontsize=fontsize)
ax.set_ylabel('PV kWp / unit',fontsize=fontsize)
ax.set_xlabel('Storage kWh / unit',fontsize=fontsize)
# Set Tick Lablels
# ----------------
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text2D(0,1.07,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text2D(0,1.0,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text2D(0,0.93,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'scm_3D'  +'_'+ 'site_'  +site + '_'+ strategy + '_' + arrangement +'.jpg'
plotFile = os.path.join(plot_bau,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()

In [10]:
# 3D Plots of delta_SC and delta_SS
# ---------------------------------


# ----------------------
# reindex and stack delta SC
# ----------------------
# dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
dfu = dfx['delta_sc'].unstack()
cols =dfu.columns
dfu.columns=cols
dfv=dfu.transpose()

# ------------------------------
# Plot 3D delta_SC vs kW peak and kWh 
# ------------------------------
dfplot = dfv.copy()
x = dfplot.columns
y = dfplot.index
X,Y = np.meshgrid(x,y)
Z = dfplot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z)

# Set Axes limits
# ---------------
ax.set_zlim(0,20)
maxz=20

#Set rotation angle 
#------------------
ax.view_init(azim=150)
#ax.view_init(elev=50)

# Label Axes
# ----------
ax.set_zlabel('Increased Self-Consumption %', fontsize=fontsize)
ax.set_ylabel('PV kWp / unit',fontsize=fontsize)
ax.set_xlabel('Storage kWh / unit',fontsize=fontsize)
# Set Tick Lablels
# ----------------
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text2D(0,1.07,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text2D(0,1.0,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text2D(0,0.93,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'delta_sc_3D'  +'_'+ 'site_'  +site + '_'+ strategy + '_' + arrangement +'.jpg'
plotFile = os.path.join(plot_bau,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()

# ----------------------
# reindex and stack delta SS
# ----------------------
# dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
dfu = dfx['delta_ss'].unstack()
cols =dfu.columns
dfu.columns=cols
dfv=dfu.transpose()

# ------------------------------
# Plot 3D delta_SS vs kW peak and kWh 
# ------------------------------
dfplot = dfv.copy()
x = dfplot.columns
y = dfplot.index
X,Y = np.meshgrid(x,y)
Z = dfplot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z)

# Set Axes limits
# ---------------
ax.set_zlim(0,15)
maxz=20

#Set rotation angle 
#------------------
ax.view_init(azim=210)
#ax.view_init(elev=50)

# Label Axes
# ----------
ax.set_zlabel('Increased Self-Sufficiency %', fontsize=fontsize)
ax.set_ylabel('PV kWp / unit',fontsize=fontsize)
ax.set_xlabel('Storage kWh / unit',fontsize=fontsize)
# Set Tick Lablels
# ----------------
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.yaxis.set_major_locator(plt.MaxNLocator(4))

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text2D(0,1.07,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text2D(0,1.0,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text2D(0,0.93,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'delta_ss_3D'  +'_'+ 'site_'  +site + '_'+ strategy + '_' + arrangement +'.jpg'
plotFile = os.path.join(plot_bau,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()



In [11]:
# 2D Plots of delta_SC and delta_SS vs kWp and kWh
# ------------------------------------------------
# ----------------------
# reindex and stack delta SC
# ----------------------
# dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
dfu = dfx['delta_sc'].unstack()
cols =dfu.columns
dfu.columns=cols
dfv=dfu.transpose()

# --------------------
# Plot delta_SC vs kW peak
# --------------------
dfplot = dfv.copy()
cols = dfplot.columns
cols2 = [str(round(c,1)) + ' kWh / unit' for c in cols]
dfplot.columns = cols2
fig, ax = plt.subplots()
# Legend outside box and resize chart to fit
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
ax.set_ylim(0,20)
maxy=20
ax.set_ylabel('Increased Self-Consumption %')
ax.set_xlabel('PV kWp / unit')

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'delta_sc_kWp'  +'_'+ 'site_' + site + '_'+ arrangement + '_' + strategy +'.jpg'
plotFile = os.path.join(plot_en,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()


# ----------------------
# reindex and stack delta SC
# ----------------------
# dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
dfu = dfx['delta_ss'].unstack()
cols =dfu.columns
dfu.columns=cols
dfv=dfu.transpose()
# ----------------
# Plot delta_SS vs kWh
# ----------------
dfplot = dfv.copy().T
cols = dfplot.columns
cols2 = [str(round(c,1)) + ' kWp / unit' for c in cols]
dfplot.columns = cols2
fig, ax = plt.subplots()
# Legend outside box and resize chart to fit
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
ax.set_ylim(0,11)
maxy=10
ax.set_ylabel('Increased Self-Sufficiency %')
ax.set_xlabel('Storage kWh / unit')

# Label Parameters
# ----------------
pa = arrangement
pt = site_tag[site]
ps = strategy
ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

pname = 'delta_ss_kWh'  +'_'+ 'site_' + site + '_'+ arrangement + '_'+ strategy +'.jpg'
plotFile = os.path.join(plot_en,pname)
plt.savefig(plotFile,dpi=1000)
plt.close()
